In [19]:
"""
Ref.: A divide-and-conquer algorithm for quantum state preparation
https://arxiv.org/abs/2008.01511
"""
import pennylane as qml
from pennylane import numpy as np

class bin_heap:
    size = None
    values = None

    def __init__(self, values):
        self.size = len(values)
        self.values = values

    def parent(self, key):
        return int((key-0.5)/2)

    def left(self, key):
        return int(2 * key + 1)

    def right(self, key):
        return int(2 * key + 2)

    def root(self):
        return 0

    def __getitem__(self, key):
        return self.values[key]


class Encoding:
    quantum_data = None
    classical_data = None
    num_qubits = None
    heap = None
    output_qubits = []

    def __init__(self, q, input_vector, encode_type='amplitude_encoding'):
        self.output_qubits = []
        if encode_type == 'dc_amplitude_encoding':
            self.dc_amplitude_encoding(q, input_vector)
        elif encode_type == 'dc_amplitude_encoding_orthonormal_ancillary':
            self.dc_amplitude_encoding_orthonormal_ancillary(q, input_vector)

    @staticmethod
    def _recursive_compute_beta(input_vector, betas):
        if len(input_vector) > 1:
            new_x = []
            beta = []
            for j in range(0, len(input_vector), 2):
                norm = np.sqrt(input_vector[j] ** 2 + input_vector[j + 1] ** 2)
                new_x.append(norm)
                if norm == 0:
                    beta.append(0)
                else:
                    if input_vector[j] < 0:
                        beta.append(2 * np.pi - 2 * np.arcsin(input_vector[j + 1] / norm))
                    else:
                        beta.append(2 * np.arcsin(input_vector[j + 1] / norm))
            Encoding._recursive_compute_beta(new_x, betas)
            betas.append(beta)

    @staticmethod
    def _recursive_compute_beta_z(input_vector, betas):
        if len(input_vector) > 1:
            new_x = []
            beta = []
            for j in range(0, len(input_vector), 2):
                new_x.append((input_vector[j+1]+input_vector[j])/2)
                beta.append(input_vector[j+1]-input_vector[j])
                
            Encoding._recursive_compute_beta_z(new_x, betas)
            betas.append(beta)

    def dc_amplitude_encoding_orthonormal_ancillary(self, q, input_vector):
        self.dc_amplitude_encoding(q, input_vector)
        
        q_ortho = q[self.num_qubits:]
        n = len(self.output_qubits)
        self.num_qubits += n
        for i in range(n):
            qml.CNOT(wires=[self.output_qubits[i], q_ortho[i]])

    def dc_amplitude_encoding(self, q, input_vector):
        self.num_qubits = int(len(input_vector))-1
        self.quantum_data = q[:self.num_qubits]
        betas_y = []
        betas_z = []

        newx = np.copy(input_vector)
        
        Encoding._recursive_compute_beta(np.abs(newx), betas_y)
        Encoding._recursive_compute_beta_z(np.angle(newx), betas_z)
                                         
        self._dc_generate_circuit(betas_y, betas_z, self.quantum_data)

    def _dc_generate_circuit(self, betas_y, betas_z, quantum_input):
        k = 0
        for angles in betas_y:
            for angle in angles:
                qml.RY(angle, wires=quantum_input[k])
                k += 1

        k = 0
        for angles in betas_z:
            for angle in angles:
                qml.RZ(angle, wires=quantum_input[k])
                k += 1
                
        self.heap = bin_heap(quantum_input)
        my_heap = self.heap

        last = my_heap.size - 1
        actual = my_heap.parent(last)
        level = my_heap.parent(last)
        while actual >= 0:
            left_index = my_heap.left(actual)
            right_index = my_heap.right(actual)
            while right_index <= last:
                qml.CSWAP(wires=[my_heap[actual], my_heap[left_index], my_heap[right_index]])
                left_index = my_heap.left(left_index)
                right_index = my_heap.left(right_index)
            actual -= 1
            if level != my_heap.parent(actual):
                level -= 1
                
        # set output qubits
        next_index = 0
        while next_index < my_heap.size:
            self.output_qubits.append(quantum_input[next_index])
            next_index = my_heap.left(next_index)

In [20]:
"""
Circuits used for testing.
"""
qml.enable_tape()

dev_amp = qml.device("default.qubit", wires=3)
dev_dc = qml.device("default.qubit", wires=7)

"""
Amplitude encoding, used for comparison.
"""
@qml.qnode(dev_amp)
def test_amp(state_vector=None):
    q=range(3)
    
    qml.templates.AmplitudeEmbedding(state_vector, wires=q)
    
    return qml.probs(wires=q)
"""
Divide-and-conquer encoding.
"""
@qml.qnode(dev_dc)
def test_dcsp(state_vector=None):

    encode = Encoding(range(10), state_vector, 'dc_amplitude_encoding')
    q = encode.output_qubits 
    
    return qml.probs(wires=q)

@qml.qnode(dev_dc)
def test_dcsp_state(state_vector=None):

    encode = Encoding(range(10), state_vector, 'dc_amplitude_encoding')
    q = encode.output_qubits 
    
    return qml.state()

In [21]:
"""
State to be loaded.
"""
state_vector_1 = [np.sqrt(0.03)*np.exp(1.0j), np.sqrt(0.07)*np.exp(1.1j), np.sqrt(0.15)*np.exp(1.22j), np.sqrt(0.05)*np.exp(1.3j), np.sqrt(0.1)*np.exp(1.4j), np.sqrt(0.3)*np.exp(1.55j), np.sqrt(0.2)*np.exp(1.6j), np.sqrt(0.1)*np.exp(1.7j) ]

In [22]:
"""
Amplitude enconding, to be used as a reference.
"""
print("Amplitude encoding:")
print("probabilities:")
print(test_amp(state_vector=state_vector_1))

Amplitude encoding:
probabilities:
[0.03 0.07 0.15 0.05 0.1  0.3  0.2  0.1 ]


In [23]:
"""
Dcsp state preparation.
"""
print("Dcsp encoding:")
print("probabilities:")
print(test_dcsp(state_vector=state_vector_1))

Dcsp encoding:
probabilities:
[0.03 0.07 0.15 0.05 0.1  0.3  0.2  0.1 ]


In [24]:
print("Dcsp circuit diagram:")
print(test_dcsp.draw())

Dcsp circuit diagram:
 0: ──RY(1.98)──RZ(0.407)────────────────╭C─────╭C─────╭┤ Probs 
 1: ──RY(1.91)──RZ(0.21)──────────╭C─────├SWAP──│──────├┤ Probs 
 2: ──RY(1.43)──RZ(0.175)──╭C─────│──────╰SWAP──│──────│┤       
 3: ──RY(1.98)──RZ(0.1)────│──────├SWAP─────────├SWAP──╰┤ Probs 
 4: ──RY(1.05)──RZ(0.08)───│──────╰SWAP─────────│───────┤       
 5: ──RY(2.09)──RZ(0.15)───├SWAP────────────────╰SWAP───┤       
 6: ──RY(1.23)──RZ(0.1)────╰SWAP────────────────────────┤       

